In [77]:
import numpy as np
import pandas as pd
import os

In [78]:
dir_path ='./data/raw/excel'

In [79]:
def read_excel_data(file_name):
    dat = pd.read_excel(file_name, na_filter = False, convert_float = False)

    # Setting the column names
    dat.columns = ['num', 'flight', 'operator', 'aircraft', 'frequency', 'from', 'to_time', 'to', 
                   'from_time', 'eff_from', 'eff_to']  

    # Altering the column positions
    dat.insert(6, 'from_time', dat.pop('from_time'))
    dat.insert(7, 'to', dat.pop('to'))
    
    return dat

In [80]:
file_name = 'Vistara.xlsx'

file_name = dir_path + '/' + file_name
dat = read_excel_data(file_name)
dat.head()

,num,flight,operator,aircraft,frequency,from,from_time,to,to_time,eff_from,eff_to
0,Ahmedabad,,,,,,,,,,
1,1,UK 939,UK,A 320,Daily,DEL,,,6:25,2019-11-25 00:00:00,2020-03-28 00:00:00
2,2,UK 936,UK,A 320,Daily,,7:05,DEL,,2019-11-25 00:00:00,2020-03-28 00:00:00
3,3,UK 959,UK,A 320,Daily,DEL,,,8:00,2019-10-27 00:00:00,2020-03-28 00:00:00
4,4,UK 946,UK,A 320,Daily,,8:40,DEL,,2019-10-27 00:00:00,2020-03-28 00:00:00


In [81]:
dat.shape

(554, 11)

In [82]:
def wrangle_rows(df):
    # dropping superflous header rows
    
    r_drop_idx = [i for i in range(df.shape[0]) if ((not isinstance(df.iloc[i, 0], float)) and 
                                                    (df.iloc[i, 4] == 'Frequency'))]
      
    df = df.drop(r_drop_idx, axis = 0)
    
    df = df.reset_index() # resetting the index after dropping rows
    df = df.drop('index', axis = 1) # dropping the previous index
    
    # dropping city names subheader rows
    r_drop_idx = [i for i in range(df.shape[0]) if isinstance(df.iloc[i, 0], str)]
    
    for i in range(len(r_drop_idx)-1):
        df.iloc[r_drop_idx[i] + 1: r_drop_idx[i+1], 0] = df.iloc[r_drop_idx[i], 0]
    
    # Separately for the last sub-head in the list
    df.iloc[r_drop_idx[-1] + 1: df.shape[0], 0] = df.iloc[r_drop_idx[-1], 0]
        
    df = df.drop(r_drop_idx, axis = 0) 
    df = df.reset_index() # resetting the index after dropping rows
    df = df.drop('index', axis = 1) # dropping the previous index
    
    df = df.rename(columns = {'num': 'station'}, errors = 'raise')
    
    return df

In [83]:
dat = wrangle_rows(dat)
dat.shape

(515, 11)

In [84]:
def wrangle_cols(df):
    idx = df.loc[:,'from'] == ''
    df.loc[idx, 'from'] = df.loc[idx, 'station']

    idx = df.loc[:, 'to'] == ''
    df.loc[idx, 'to'] = df.loc[idx, 'station']

    df = df.drop(['station'], axis = 1)

    return df

In [85]:
dat = wrangle_cols(dat)
dat.head()

,flight,operator,aircraft,frequency,from,from_time,to,to_time,eff_from,eff_to
0,UK 939,UK,A 320,Daily,DEL,,Ahmedabad,6:25,2019-11-25 00:00:00,2020-03-28 00:00:00
1,UK 936,UK,A 320,Daily,Ahmedabad,7:05,DEL,,2019-11-25 00:00:00,2020-03-28 00:00:00
2,UK 959,UK,A 320,Daily,DEL,,Ahmedabad,8:00,2019-10-27 00:00:00,2020-03-28 00:00:00
3,UK 946,UK,A 320,Daily,Ahmedabad,8:40,DEL,,2019-10-27 00:00:00,2020-03-28 00:00:00
4,UK 979,UK,A 320,123456,DEL,,Ahmedabad,10:20,2019-10-28 00:00:00,2020-03-28 00:00:00


In [86]:
def fix_city_spelling(df):
    city_names = {'Delhi' : 'New Delhi', 'Bhubaneshwar' : 'Bhubaneswar',
                  'Trivandrum' : 'Thiruvananthapuram', 'Pondicherry' : 'Puducherry',
                  'Porbander' : 'Porbandar', 'Tirupathi' : 'Tirupati',
                  'Tuticorin' : 'Thoothukudi', 'Vizag' : 'Visakhapatnam',
                  'Cuddapah': 'Kadapa', 'Jalgoan' : 'Jalgaon',
                  'Rajamundry' : 'Rajahmundry', 'Aizawal' : 'Aizawl',
                  'Trichy' : 'Tiruchirappally', 'Bathinda' : 'Bhatinda',
                  'Passighat' : 'Pasighat'}
    
    missing_city_codes = []
    
    for i in range(df.shape[0]):
            if df['from'][i] in list(city_names.keys()): df['from'][i] = city_names[df['from'][i]]
            if df['to'][i] in list(city_names.keys()): df['to'][i] = city_names[df['to'][i]]
            
    return df

In [87]:
dat = fix_city_spelling(dat)
dat.head()

,flight,operator,aircraft,frequency,from,from_time,to,to_time,eff_from,eff_to
0,UK 939,UK,A 320,Daily,DEL,,Ahmedabad,6:25,2019-11-25 00:00:00,2020-03-28 00:00:00
1,UK 936,UK,A 320,Daily,Ahmedabad,7:05,DEL,,2019-11-25 00:00:00,2020-03-28 00:00:00
2,UK 959,UK,A 320,Daily,DEL,,Ahmedabad,8:00,2019-10-27 00:00:00,2020-03-28 00:00:00
3,UK 946,UK,A 320,Daily,Ahmedabad,8:40,DEL,,2019-10-27 00:00:00,2020-03-28 00:00:00
4,UK 979,UK,A 320,123456,DEL,,Ahmedabad,10:20,2019-10-28 00:00:00,2020-03-28 00:00:00


In [88]:
def wrangle_iata_codes(df):
    # Load the dictionary, previously created, mapping city names to corresponding IATA codes
    
    import pickle
    
    with open('./data/processed/dicts/city_to_codes_dict.txt', 'rb') as handle:
        city_to_codes = pickle.loads(handle.read())
    
    # To capture city names in the df for which an equivalen IATA code is missing
    missing_city_names = []
    
    idx = [i for i in range(df.shape[0]) if not df['from'][i].isupper()]
    for i in idx: 
        try: 
            df['from'][i] = city_to_codes[df['from'][i]]
        except KeyError:
            if df['from'][i] not in missing_city_names:
                missing_city_names.append(df['from'][i])
        
    idx = [i for i in range(df.shape[0]) if not df['to'][i].isupper()]
    for i in idx: 
        try:
            df['to'][i] = city_to_codes[df['to'][i]]
        except KeyError:
            if df['to'][i] not in missing_city_names:
                missing_city_names.append(df['to'][i])
        
    return missing_city_names, df

In [89]:
missing_city_names, dat = wrangle_iata_codes(dat)

print('The list of city names either with wrong spelling or missing IATA codes\n', missing_city_names)

The list of city names either with wrong spelling or missing IATA codes
 []


In [90]:
missing_city_names, dat = wrangle_iata_codes(dat)
print('The list of city names either with wrong spelling or missing IATA codes\n', missing_city_names)
dat.head()

The list of city names either with wrong spelling or missing IATA codes
 []


,flight,operator,aircraft,frequency,from,from_time,to,to_time,eff_from,eff_to
0,UK 939,UK,A 320,Daily,DEL,,AMD,6:25,2019-11-25 00:00:00,2020-03-28 00:00:00
1,UK 936,UK,A 320,Daily,AMD,7:05,DEL,,2019-11-25 00:00:00,2020-03-28 00:00:00
2,UK 959,UK,A 320,Daily,DEL,,AMD,8:00,2019-10-27 00:00:00,2020-03-28 00:00:00
3,UK 946,UK,A 320,Daily,AMD,8:40,DEL,,2019-10-27 00:00:00,2020-03-28 00:00:00
4,UK 979,UK,A 320,123456,DEL,,AMD,10:20,2019-10-28 00:00:00,2020-03-28 00:00:00


In [91]:
def drop_dummy_scheds(df):
    idx = [i for i in range(df.shape[0]) if df['eff_from'][i] == df['eff_to'][i]]
    df = df.drop(idx, axis = 0)
    df = df.reset_index()
    df = df.drop(['index'], axis = 1)
    
    return df

In [92]:
dat = drop_dummy_scheds(dat)
dat.shape

(486, 10)

In [93]:
def drop_noncurrent_scheds(df, tz):
    today = datetime.now(tz).date()
    idx = [i for i in range(df.shape[0]) 
           if (today <=  datetime.date(df['eff_from'][i])) or (today >= datetime.date(df['eff_to'][i]))]
    df = df.drop(idx, axis = 0)
    df = df.reset_index()
    df = df.drop('index', axis = 1)
    
    return df

In [94]:
from datetime import datetime
import pytz

tz = pytz.timezone('Asia/Calcutta')
dat = drop_noncurrent_scheds(dat, tz)
dat.shape

(462, 10)

In [95]:
def wrangle_frequency(df):
# Convert frequency into a string of numbers
    for i in range(df.shape[0]):
        if isinstance(df.frequency[i], str):
            df.frequency[i] = '1234567'
        else:
            df.frequency[i] = str(int(df.frequency[i]))

# Convert string of numbers into list of numbers
        df.frequency[i] = list(df.frequency[i]) # Split str
        df.frequency[i] = list(map(int, df.frequency[i])) # Convert from str to int
    
    return df

In [96]:
dat = wrangle_frequency(dat)

dat.head()

,flight,operator,aircraft,frequency,from,from_time,to,to_time,eff_from,eff_to
0,UK 939,UK,A 320,"[1, 2, 3, 4, 5, 6, 7]",DEL,,AMD,6:25,2019-11-25 00:00:00,2020-03-28 00:00:00
1,UK 936,UK,A 320,"[1, 2, 3, 4, 5, 6, 7]",AMD,7:05,DEL,,2019-11-25 00:00:00,2020-03-28 00:00:00
2,UK 959,UK,A 320,"[1, 2, 3, 4, 5, 6, 7]",DEL,,AMD,8:00,2019-10-27 00:00:00,2020-03-28 00:00:00
3,UK 946,UK,A 320,"[1, 2, 3, 4, 5, 6, 7]",AMD,8:40,DEL,,2019-10-27 00:00:00,2020-03-28 00:00:00
4,UK 979,UK,A 320,"[1, 2, 3, 4, 5, 6]",DEL,,AMD,10:20,2019-10-28 00:00:00,2020-03-28 00:00:00


In [97]:
def get_yes_no_answer():
    
    '''
    Gets user to input either 'y' or 'n' or 'yes' or 'no' or their case variations
    '''
    while True:
        reply = str(input('Combine?: (y/n): ')).lower().strip()
        
        if reply == 'y' or reply == 'n':
            break
        else:
            print("Please select 'yes' or 'no'")
            continue
    
    return reply

In [98]:
def combine_pairs(df, human_intel = 'n'):
    drop_idx = []
    
    def combine_pairs_func(i, j, drop_idx):
        drop_idx.append(j)
        
        if df.to_time[i] == '': 
            df.to_time[i] = df.to_time[j]
        else: 
            df.from_time[i] = df.from_time[j]
            
        return drop_idx
        
    
    for i in range(df.shape[0] - 1):
        if i not in drop_idx:
            for j in range(i+1, df.shape[0]):
                if j not in drop_idx:
                    
                    if human_intel == 'n':
                        if df.flight[i] == df.flight[j] and df.frequency[i] == df.frequency[j]:
                            if df['from'][i] == df['from'][j] and df.to[i] == df.to[j]:
                                drop_idx = combine_pairs_func(i, j, drop_idx)
                                
                    else:
                        if df.flight[i] == df.flight[j] and df['from'][i] == df['from'][j]:
                            if df.to[i] == df.to[j]:
                                if df.to_time[i] == '' and df.from_time[j] == '' and df.from_time[i] != '' and df.to_time[j] != '':
                                    display('Possibly Pairs?')
                                    display(df.iloc[i, :8], df.iloc[j, :8])
                                    
                                    reply = get_yes_no_answer()
                                    
                                    if reply == 'y':
                                        drop_idx = combine_pairs_func(i, j, drop_idx)
                                    
                                elif df.to_time[i] != '' and df.from_time[j] != '' and df.from_time[i] == '' and df.to_time[j] == '':
                                    display('Pairs?')
                                    display(df.iloc[i, :8], df.iloc[j, :8])
                                    
                                    reply = get_yes_no_answer()
                                    
                                    if reply == 'y':
                                        drop_idx = combine_pairs_func(i, j, drop_idx)
                            
    
    df = df.drop(drop_idx, axis = 0)
    df = df.reset_index()
    df = df.drop('index', axis = 1)
    
    return(df)

In [99]:
dat = combine_pairs(dat)

dat.shape

(292, 10)

In [100]:
def combine_freq(df, human_intel = 'n'):
    
    '''
    Combine multiple schedules where the schedules in the set differ only in their frequencies. This requires no 
    human intervention.

    Combine multiple schedules where the schedules in the set differ not only in their frequencies but vary 
    slightly in their <from_time> (or <to_time>) values. This requires human intervention
    '''
    
    def combine_freq_func(i, j, drop_idx):
        drop_idx.append(j)
        df.frequency[i] = df.frequency[i] + df.frequency[j]
        df.frequency[i] = list(set(df.frequency[i]))
        return drop_idx
        
    drop_idx = []    
    
    for i in range(df.shape[0] - 1):
        if i not in drop_idx:
            for j in range(i+1, df.shape[0]):
                if j not in drop_idx:
                    
                    if human_intel == 'n':
                        
                        if df.flight[i] == df.flight[j] and df['from'][i] == df['from'][j] and df.to[i] == df.to[j]:
                            if df.to_time[i] == df.to_time[j] and df.from_time[i] == df.from_time[j]:
                                drop_idx = combine_freq_func(i, j, drop_idx)
                    
                    else:
                        
                        if df.flight[i] == df.flight[j] and df['from'][i] == df['from'][j] and df.to[i] == df.to[j]:
                            if df.to_time[i] == df.to_time[j] or df.from_time[i] == df.from_time[j]:
                                
                                display(df.iloc[i, :8], '\n', df.iloc[j, :8])
                                display('Probably the same flight.')

                                reply = get_yes_no_answer()
    
                                if reply == 'y':
                                    drop_idx = combine_freq_func(i, j, drop_idx)
      
    df = df.drop(drop_idx, axis = 0)
    df = df.reset_index()
    df = df.drop('index', axis = 1)
    
    return(df)

In [101]:
dat = combine_freq(dat)

dat.shape

(270, 10)

In [102]:
dat = combine_pairs(dat)

dat.shape

(253, 10)

In [103]:
dat = combine_freq(dat, 'y')
dat = combine_pairs(dat)
dat.shape

flight       UK 863
operator         UK
aircraft      A 320
frequency       [2]
from            BOM
from_time          
to              BLR
to_time       10:55
Name: 29, dtype: object

'\n'

flight                   UK 863
operator                     UK
aircraft                  A 320
frequency    [1, 3, 4, 5, 6, 7]
from                        BOM
from_time                      
to                          BLR
to_time                   11:10
Name: 30, dtype: object

'Probably the same flight.'

Combine?: (y/n): y


...

'\n'

flight       UK 874
operator         UK
aircraft      B 738
frequency       [2]
from            HYD
from_time      8:20
to              BOM
to_time            
Name: 199, dtype: object

'Probably the same flight.'

Combine?: (y/n): y


(247, 10)

In [104]:
dat = combine_pairs(dat, 'y')

dat.shape

'Pairs?'

flight                      UK 845
operator                        UK
aircraft                     B 738
frequency    [1, 2, 3, 4, 5, 6, 7]
from                           BOM
from_time                         
to                             BLR
to_time                       7:35
Name: 22, dtype: object

flight                   UK 845
operator                     UK
aircraft                  B 738
frequency    [1, 2, 3, 4, 5, 6]
from                        BOM
from_time                  6:00
to                          BLR
to_time                        
Name: 226, dtype: object

Combine?: (y/n): y


'Possibly Pairs?'

...

'Pairs?'

flight                UK 930
operator                  UK
aircraft               A 320
frequency    [1, 2, 3, 4, 5]
from                     BOM
from_time                   
to                       DEL
to_time                 9:35
Name: 110, dtype: object

flight                      UK 930
operator                        UK
aircraft                     A 320
frequency    [1, 2, 3, 4, 5, 6, 7]
from                           BOM
from_time                     7:30
to                             DEL
to_time                           
Name: 227, dtype: object

Combine?: (y/n): y


(241, 10)

In [109]:
trash_idx = [i for i in range(dat.shape[0]) if dat.to_time[i] == '' or dat.from_time[i] == '']

dat = dat.drop(trash_idx, axis = 0)
dat = dat.reset_index()
dat = dat.drop('index', axis = 1)
dat.shape

(192, 10)

In [111]:
foo = dat.copy()
foo.dtypes

flight       object
operator     object
aircraft     object
frequency    object
from         object
from_time    object
to           object
to_time      object
eff_from     object
eff_to       object
dtype: object

In [165]:
import datetime
datetime_cols = ['from_time', 'to_time']
bar = datetime.datetime.strptime(foo['to_time'][0], '%H:%M')
car = datetime.datetime.strptime(foo['from_time'][1], '%H:%M')

In [166]:
print(bar, car)
(bar>car)

1900-01-01 08:00:00 1900-01-01 08:40:00


False

In [128]:
type(foo['to_time'][0])

str

In [110]:
dat.to_excel('./data/processed/vistara.xlsx')